# Notebook: 5 KCs toy example

## Package import

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac
sys.path.append("/home/olivier/PycharmProjects/bayesian-kst/")  # for ubuntu

from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.resources_layer.exercise_family import ExerciseFamily
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
from sklearn.metrics import roc_auc_score


## Domain knowledge model
We define a set of 5 KCs (A, B, C, D and E) and the relationships that rule them ($A \longrightarrow C$, $B \longrightarrow C$, $C \longrightarrow D$, $C \longrightarrow E$). We suppose that the prerequisites $A \longrightarrow C$ and $C \longrightarrow D$ are strong while $B \longrightarrow C$ and $C \longrightarrow E$ are weak. Figure \ref{fig:symbolic_model_5_kcs} represents the domain knowledge model of this 5 KCs example.

In [2]:
# we define the KCs
KC_A = KnowledgeComponent(1, "A")
KC_B = KnowledgeComponent(2, "B")
KC_C = KnowledgeComponent(3, "C")
KC_D = KnowledgeComponent(4, "D")
KC_E = KnowledgeComponent(5, "E")

A_2_C = Link(source=KC_A, target=KC_C)
B_2_C = Link(source=KC_B, target=KC_C)
C_2_D = Link(source=KC_C, target=KC_D)
C_2_E = Link(source=KC_C, target=KC_E)

domain = Domain([KC_A, KC_B, KC_C, KC_D, KC_E], [A_2_C, B_2_C, C_2_D, C_2_E])

params = {"slip": .01, "guess":.01}

# there are 5 exercises corresponding to KC A
ex_1 = Exercise(1, KC_A, "qcm", ex_content="", params=params)
ef_1 = ExerciseFamily(1, "ex fam 1", kc=KC_A, exercise_list=[ex_1])
ex_2 = Exercise(2, KC_B, "qcm", ex_content="", params=params)
ef_2 = ExerciseFamily(2, "ex fam 2", kc=KC_B, exercise_list=[ex_2])
ex_3 = Exercise(3, KC_C, "qcm", ex_content="", params=params)
ef_3 = ExerciseFamily(3, "ex fam 3", kc=KC_C, exercise_list=[ex_3])
ex_4 = Exercise(4, KC_D, "qcm", ex_content="", params=params)
ef_4 = ExerciseFamily(4, "ex fam 4", kc=KC_D, exercise_list=[ex_4])
ex_5 = Exercise(5, KC_E, "qcm", ex_content="", params=params)
ef_5 = ExerciseFamily(5, "ex fam 5", kc=KC_E, exercise_list=[ex_5])

link_strengths = {KC_A:{KC_C: 'strong'}, KC_B:{KC_C: 'weak'}, KC_C: {
    KC_A: 'strong', KC_B: 'weak', KC_D: 'strong', KC_E: 'weak'}, KC_D:{KC_C: 'strong'}, KC_E:{KC_C: 'weak'}}

learner_pool = LearnerPool(domain, link_strengths)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, .1)
    learner_pool.set_prior(kc, .1)
    learner_pool.set_slip(kc, .1)
    learner_pool.set_guess(kc, .25)
    learner_pool.set_forget(kc, 0)

## Use case 1: learner lacking of prerequisite mastering
Let define a learner $L_0$ as following: we suppose $L_0$ greatly masters KC $A$ but because he doesn't master KC $B$, he can't correctly answer to exercises on KC $C$ and, therefore, on KC $D$ and $E$ neither.

In [3]:
learner = Learner(1, None)
learner_pool.add_learner(learner)
priors = learner.get_priors()

knowledge_state = learner.predict_next_step(priors, evaluation=[KC_A, True], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.6290655221426382, 'B': 0.34519582090800144, 'C': 0.38919971572698636, 'D': 0.15621080794849643, 'E': 0.14266603494885874, 'eval(A)': 0.6588925893927149, 'eval(B)': 0.4743772835902009, 'eval(C)': 0.5029798152225412, 'eval(D)': 0.3515370251665227, 'eval(E)': 0.3427329227167582}


In [4]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_B, False], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.7404120719900182, 'B': 0.1521183075937299, 'C': 0.3335316507485333, 'D': 0.10633703241854849, 'E': 0.10850215201226064, 'eval(A)': 0.7312678467935119, 'eval(B)': 0.3488768999359244, 'eval(C)': 0.46679557298654656, 'eval(D)': 0.31911907107205645, 'eval(E)': 0.32052639880796946}


In [5]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_C, False], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.7561314697309712, 'B': 0.19472664917168125, 'C': 0.08426922268628156, 'D': 0.03032696940085968, 'E': 0.050178226377396676, 'eval(A)': 0.7414854553251312, 'eval(B)': 0.37657232196159285, 'eval(C)': 0.304774994746083, 'eval(D)': 0.26971253011055885, 'eval(E)': 0.28261584714530785}


In [6]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_C, False], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.7558563362470011, 'B': 0.19625020648260139, 'C': 0.020700783434930748, 'D': 0.006805073698701935, 'E': 0.02049239309658017, 'eval(A)': 0.7413066185605508, 'eval(B)': 0.37756263421369096, 'eval(C)': 0.263455509232705, 'eval(D)': 0.25442329790415624, 'eval(E)': 0.26332005551277715}


## Use case 2: learner already mastering the most advanced KCs
Let define another learner $L_2$ as following: we suppose $L_1$ has already studied the domain knowledge and therefore greatly masters every KCs but KC $E$ he hasn't studied yet.

In [7]:
learner = Learner(2, None)
learner_pool.add_learner(learner)
priors = learner.get_priors()

knowledge_state = learner.predict_next_step(priors, evaluation=[KC_C, True], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.4940130432260898, 'B': 0.40685083623620877, 'C': 0.4871761869969719, 'D': 0.1904108768445694, 'E': 0.16853388248370393, 'eval(A)': 0.5711084780969584, 'eval(B)': 0.5144530435535357, 'eval(C)': 0.5666645215480317, 'eval(D)': 0.37376706994897013, 'eval(E)': 0.35954702361440755}


In [8]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_D, False], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.740187531926506, 'B': 0.6313026792255861, 'C': 0.5405815426767001, 'D': 0.03275546394716906, 'E': 0.16853388248370393, 'eval(A)': 0.7311218957522289, 'eval(B)': 0.660346741496631, 'eval(C)': 0.6013780027398551, 'eval(D)': 0.2712910515656599, 'eval(E)': 0.35954702361440755}


In [9]:
knowledge_state = learner.predict_next_step(knowledge_state, evaluation=[KC_E, False], pred_mode='one_kc')
print(knowledge_state)

{'A': 0.8670312493404424, 'B': 0.771304758332246, 'C': 0.5424584310303202, 'D': 0.03275546394716906, 'E': 0.030087924825804604, 'eval(A)': 0.8135703120712876, 'eval(B)': 0.7513480929159599, 'eval(C)': 0.6025979801697081, 'eval(D)': 0.2712910515656599, 'eval(E)': 0.2695571511367729}
